Original work by https://www.kaggle.com/inversion/processing-bson-files
Add product_to_category in the definition of process for the multiprocessing, so it works for the Win10.  

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import io
import os
import bson
import matplotlib.pyplot as plt
from skimage.data import imread 
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
data_dir='../input'
data = bson.decode_file_iter(open(os.path.join(data_dir,'train_example.bson'),'rb'))

In [ ]:
product_to_category=dict()
for c,d in enumerate(data):
    product_id = d['_id']
    category_id = d['category_id']
    product_to_category[product_id] = category_id
    for e,pic in enumerate(d['imgs']):
        picture = imread(io.BytesIO(pic['picture']))

In [ ]:
product_to_category = pd.DataFrame.from_dict(product_to_category,orient='index')

In [ ]:
product_to_category.index.name='_id'
product_to_category.rename(columns={0:'category_id'},inplace=True)

In [ ]:
product_to_category.head()

In [ ]:
import multiprocessing as mp

In [ ]:
NCORE =  2
manager = mp.Manager()



In [ ]:
product_to_category = manager.dict()

In [ ]:
def process(q, iolock, product_to_category):
    while True:
        d=q.get()
        if d is None:
            break
        product_id = d['_id']
        category_id = d['category_id']
        product_to_category[product_id] = category_id
        for e,pic in enumerate(d['imgs']):
            picture = imread(io.BytesIO(pic['picture']))
            

In [ ]:
data = bson.decode_file_iter(open(os.path.join(data_dir,'train_example.bson'),'rb'))
q=mp.Queue(maxsize=NCORE)
iolock=mp.Lock()
pool = mp.Pool(NCORE, initializer=process, initargs=(q, iolock, product_to_category))
for c,d in enumerate(data):
    q.put(d)
    
for _ in range(NCORE):
    q.put(None)
pool.close()
pool.join()

In [ ]:
product_to_category=dict(product_to_category)

In [ ]:
product_to_category